<a href="https://colab.research.google.com/github/Keba462/Csi408_/blob/master/DescionForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_decision_forests

In [2]:
!pip install wurlitzer

## Importing libraries

In [3]:
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

try:
  from wurlitzer import sys_pipes
except:
  from colabtools.googlelog import CaptureLog as sys_pipes

from IPython.core.magic import register_line_magic
from IPython.display import Javascript

### Load the dataset and convert it in a tf.Dataset

In [5]:
col_names = ['age', 'body temperature', 'Dry Cough', 'sore throat', 'weakness', 'breathing problems', 'drowsiness', 'pain in chest ', 'travel history to infected countries', 'diabetes', 'heart disease', 'lung disease', 'stroke or reduced immunity', 'symptoms progressed', 'high blood pressure', 'kidney disease', 'change in appetite', 'loss of sense of smell', 'label']
# Load a dataset into a Pandas Dataframe.
corona=pd.read_excel('/content/drive/MyDrive/Project 408/new.xlsx', header=None, names=col_names)

# Display the first 3 examples.
corona.head(3)

,age,body temperature,Dry Cough,sore throat,weakness,breathing problems,drowsiness,pain in chest,travel history to infected countries,diabetes,heart disease,lung disease,stroke or reduced immunity,symptoms progressed,high blood pressure,kidney disease,change in appetite,loss of sense of smell,label
0,20,98.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19,99.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,55,102.0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1


Next split the dataset into training and testing:

In [8]:
label = "label"

# Split the dataset into a training and a testing dataset.

def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(corona)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

86 examples in training, 41 examples for testing.


convert the pandas dataframe (`pd.Dataframe`) into tensorflow datasets (`tf.data.Dataset`):

In [9]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)

/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core.py:2036: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


### Train the model

In [11]:
##%set_cell_height 300

# Specify the model.
model_1 = tfdf.keras.RandomForestModel()

# Optionally, add evaluation metrics.
model_1.compile(
    metrics=["accuracy"])

# Train the model.
# "sys_pipes" is optional. It enables the display of the training logs.
with sys_pipes():
  model_1.fit(x=train_ds)

Use /tmp/tmp5kioffkp as temporary training directory
Starting reading the dataset
1/1 [==============================] - ETA: 0s
Dataset read in 0:00:04.658897
Training model
Model trained in 0:00:00.060168
Compiling model


[INFO kernel.cc:1153] Loading model from path
[INFO abstract_model.cc:1063] Engine "RandomForestGeneric" built
[INFO kernel.cc:1001] Use fast generic engine


1/1 [==============================] - 5s 5s/step
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


## Evaluate the model

In [12]:
evaluation = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

1/1 [==============================] - 1s 833ms/step - loss: 0.0000e+00 - accuracy: 0.6585

loss: 0.0000
accuracy: 0.6585


Export the model to the SavedModel format for later re-use 

In [14]:
model_1.save("/content/drive/MyDrive/Project 408/my_saved_model")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project 408/my_saved_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project 408/my_saved_model/assets
